<a href="https://colab.research.google.com/github/Luna0216/2025-DL-final-project/blob/main/ptt_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

In [ ]:


BASE_URL = "https://www.ptt.cc"
CITIES = ["Tainan", "Taichung", "Kaohsiung","HsinChuang", "BigBanciao"]
KEYWORDS = ["火災", "淹水", "颱風", "地震", "疫情", "死亡", "車禍", "身亡", "自殺"]
MAX_POSTS = 4000

headers = {
    'cookie': 'over18=1',
    'user-agent': 'Mozilla/5.0'
}

def get_article_content(article_url):
    try:
        res = requests.get(article_url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        main_content = soup.find(id='main-content')
        if main_content is None:
            return ""
        for tag in main_content.select('div, span, push'):
            tag.decompose()
        content = main_content.text.strip()
        return content
    except Exception as e:
        print(f"無法取得內文: {article_url}，錯誤：{e}")
        return ""

def get_articles_by_search(board, keyword):
    results = []
    page = 1
    while len(results) < MAX_POSTS:
        url = f"{BASE_URL}/bbs/{board}/search?page={page}&q={keyword}"
        res = requests.get(url, headers=headers)
        if res.status_code != 200:
            print(f"無法搜尋：{board} 板 {keyword}（第 {page} 頁）")
            page += 1
            break

        soup = BeautifulSoup(res.text, 'html.parser')
        articles = soup.select("div.r-ent")
        if not articles:
            break

        for entry in articles:
            title_tag = entry.select_one("div.title a")
            if not title_tag:
                continue
            title = title_tag.text.strip()
            href = title_tag["href"]
            full_url = BASE_URL + href
            results.append({"title": title, "url": full_url})
            if len(results) >= MAX_POSTS:
                break

        page += 1
    print(f"{board} 板關鍵字「{keyword}」找到 {len(results)} 則文章")
    return results

def crawl_keyword(board, keyword):
    article_list = get_articles_by_search(board, keyword)
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(get_article_content, item["url"]): item for item in article_list}
        for future in as_completed(future_to_url):
            item = future_to_url[future]
            content = future.result()
            content = content.replace('\x00', '').replace('\u0000', '')
            results.append({
                "city": board,
                "keyword": keyword,
                "title": item["title"],
                "url": item["url"],
                "content": content
            })
    return results

def remove_urls(text):
    if pd.isna(text):
        return ""
    text = text.replace('\x00', '').replace('\u0000', '')
    return re.sub(r'http[s]?://\S+', '', text)

all_data = []

for city in CITIES:
    for kw in KEYWORDS:
        print(f"\n正在處理 {city} 板 關鍵字：{kw}")
        results = crawl_keyword(city, kw)
        all_data.extend(results)

df_all = pd.DataFrame(all_data)
df_all["content"] = df_all["content"].astype(str).apply(remove_urls)




正在處理 Tainan 板 關鍵字：火災
無法搜尋：Tainan 板 火災（第 7 頁）
Tainan 板關鍵字「火災」找到 101 則文章

正在處理 Tainan 板 關鍵字：淹水
無法搜尋：Tainan 板 淹水（第 6 頁）
Tainan 板關鍵字「淹水」找到 89 則文章

正在處理 Tainan 板 關鍵字：颱風
無法搜尋：Tainan 板 颱風（第 9 頁）
Tainan 板關鍵字「颱風」找到 159 則文章

正在處理 Tainan 板 關鍵字：地震
無法搜尋：Tainan 板 地震（第 20 頁）
Tainan 板關鍵字「地震」找到 374 則文章

正在處理 Tainan 板 關鍵字：疫情
無法搜尋：Tainan 板 疫情（第 14 頁）
Tainan 板關鍵字「疫情」找到 259 則文章

正在處理 Tainan 板 關鍵字：死亡
無法搜尋：Tainan 板 死亡（第 7 頁）
Tainan 板關鍵字「死亡」找到 102 則文章

正在處理 Tainan 板 關鍵字：車禍
無法搜尋：Tainan 板 車禍（第 17 頁）
Tainan 板關鍵字「車禍」找到 306 則文章

正在處理 Tainan 板 關鍵字：身亡
無法搜尋：Tainan 板 身亡（第 3 頁）
Tainan 板關鍵字「身亡」找到 21 則文章

正在處理 Tainan 板 關鍵字：自殺
無法搜尋：Tainan 板 自殺（第 2 頁）
Tainan 板關鍵字「自殺」找到 9 則文章

正在處理 Taichung 板 關鍵字：火災
無法搜尋：Taichung 板 火災（第 1 頁）
Taichung 板關鍵字「火災」找到 0 則文章

正在處理 Taichung 板 關鍵字：淹水
無法搜尋：Taichung 板 淹水（第 1 頁）
Taichung 板關鍵字「淹水」找到 0 則文章

正在處理 Taichung 板 關鍵字：颱風
無法搜尋：Taichung 板 颱風（第 1 頁）
Taichung 板關鍵字「颱風」找到 0 則文章

正在處理 Taichung 板 關鍵字：地震
無法搜尋：Taichung 板 地震（第 1 頁）
Taichung 板關鍵字「地震」找到 0 則文章

正在處理 Taichung 板 關鍵字：疫情
無法搜尋：Taichung 板 疫情（第 1 頁）

In [ ]:
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\［.*?\］', '', text)
    text = re.sub(r'\【.*?\】', '', text)
    text = re.sub(r'\n', '', text)
    text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != "C")
    return text.strip()


In [ ]:
# # 清洗並合併欄位
df_all["text"] = df_all["title"] + "。 " + df_all["content"]
df_all["content"] = df_all["content"].astype(str).apply(clean_text)

df_all["text"] = df_all["text"].astype(str).apply(clean_text)
# # 只保留需要的欄位
# df_all = df_all[["city", "keyword", "url", "text"]]

# # 儲存為 Excel
# #df_all.to_excel("ptt_all_cities_combined.xlsx", index=False)
# print("已儲存為 ptt_all_cities_combined.xlsx")


In [ ]:
df_all_filter = df_all[df_all['text'].astype(str).apply(lambda x: len(x) <= 150)]

df_all_filter

,city,keyword,url,text
0,Tainan,火災,https://www.ptt.cc/bbs/Tainan/M.1744877347.A.8...,西港 南41鄉道火災。 看FB貼的照片應該是麒宏食品工業，還好旁邊沒什麼建築相鄰。裏面都是...
2,Tainan,火災,https://www.ptt.cc/bbs/Tainan/M.1738247446.A.B...,火災後除臭。 今天太太在家開電暖器結果屋內可能電線老舊走火燒了起來雖然即時撲滅人平安但是現在...
3,Tainan,火災,https://www.ptt.cc/bbs/Tainan/M.1738112735.A.5...,府前路一段火災。 府前路一段火災遠遠看濃煙很大 都飄到南邊空氣不是很好要經過那邊的注意--
4,Tainan,火災,https://www.ptt.cc/bbs/Tainan/M.1728902217.A.4...,南區火災。 從四鯤鯓看過去鯤鯓路102巷--
5,Tainan,火災,https://www.ptt.cc/bbs/Tainan/M.1733022519.A.B...,永康中華路麥當勞隔壁的店疑似火災。 剛經過中華路麥當勞停了一排消防車應該是隔壁的夾娃娃店疑似...
...,...,...,...,...
3494,BigBanciao,車禍,https://www.ptt.cc/bbs/BigBanciao/M.1335086906...,莒光莊敬路車禍？。 剛剛騎莒光路經過莊敬路交叉口時疑似有發生車禍一位老伯伯倒在地上動也不動臉...
3496,BigBanciao,車禍,https://www.ptt.cc/bbs/BigBanciao/M.1335085794...,華江橋又車禍了。 下班回家，台北是回板橋，堵到上橋的斜坡處了。我讓救護車過後，救護車也卡在上...
3497,BigBanciao,車禍,https://www.ptt.cc/bbs/BigBanciao/M.1333976116...,民生陸橋下的車禍。 聯結車整個翻掉看起來好像是視線不良直接撞到橋墩天雨路滑小心騎車開車阿--...
3504,BigBanciao,車禍,https://www.ptt.cc/bbs/BigBanciao/M.1324459290...,互助街新海路口車禍...。 剛剛路口發生車禍探頭出去好像是轉彎的小貨車撞到機車下雨天~天雨路...


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch
model_name = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def batch_translate(texts):
    batch = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    batch = {k: v.to(device) for k, v in batch.items()}
    translated = model.generate(**batch)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]


translated_texts = []
batch_size = 20
for i in range(0, len(df_all_filter), batch_size):
    batch = df_all_filter["text"].iloc[i:i+batch_size].tolist()
    translated_batch = batch_translate(batch)
    translated_texts.extend(translated_batch)
    print(f"完成翻譯:{i}")

df_all_filter["text_en"] = translated_texts


完成翻譯:0
完成翻譯:20
完成翻譯:40
完成翻譯:60
完成翻譯:80
完成翻譯:100
完成翻譯:120
完成翻譯:140
完成翻譯:160
完成翻譯:180
完成翻譯:200
完成翻譯:220
完成翻譯:240
完成翻譯:260
完成翻譯:280
完成翻譯:300
完成翻譯:320
完成翻譯:340
完成翻譯:360
完成翻譯:380
完成翻譯:400
完成翻譯:420
完成翻譯:440
完成翻譯:460
完成翻譯:480
完成翻譯:500
完成翻譯:520
完成翻譯:540
完成翻譯:560
完成翻譯:580
完成翻譯:600
完成翻譯:620
完成翻譯:640
完成翻譯:660
完成翻譯:680
完成翻譯:700
完成翻譯:720
完成翻譯:740
完成翻譯:760
完成翻譯:780
完成翻譯:800
完成翻譯:820
完成翻譯:840
完成翻譯:860
完成翻譯:880
完成翻譯:900
完成翻譯:920
完成翻譯:940
完成翻譯:960
完成翻譯:980
完成翻譯:1000
完成翻譯:1020
完成翻譯:1040
完成翻譯:1060
完成翻譯:1080
完成翻譯:1100
完成翻譯:1120
完成翻譯:1140
完成翻譯:1160
完成翻譯:1180
完成翻譯:1200
完成翻譯:1220
完成翻譯:1240


<ipython-input-40-127729822429>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_filter["text_en"] = translated_texts


In [ ]:
df_all_filter["text_en"]

,text_en
0,"West Harbour, south of the 41st country fire, ..."
2,"Today's wife turns on her home heater, and the..."
3,"There was a fire in front of the building, and..."
4,"A fire in the southern part of the country, lo..."
5,The store next to Mid-Walk Ludlow seems to be ...
...,...
3494,"When I was riding by the intersection, there w..."
3496,"Back home from work, Taipei went back to the b..."
3497,A car crash under the Manchuria Bridge. The wh...
3504,"There was a car accident at the intersection, ..."


<ipython-input-48-858854496f86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_filter['target'] = 1


In [ ]:
df_all_filter = df_all_filter[["text_en"]]
df_all_filter.to_csv("disater_ptt.csv", index=False)